### **1. DATA COLLECTION**

In [2]:
!pip install bs4
!pip install selenium
!pip install forex-python
!pip install geopy

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=a0d84d13176ecd7ee36b4e10f2c6c0d7307f665af4ad371a543beb98f299e8bb
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.3/448.3 kB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 3.6 MB/s eta 0:00:00


In [93]:
#import the required packages
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import heapq
import nltk
from nltk.stem import *
from nltk.corpus import stopwords
import re
from collections import Counter
from functools import reduce
import numpy as np
import random



#### 1.1. Get the list of master's degree courses

In [4]:
flag = False # run only once

if flag:
    # URL of the website
    url = 'https://www.findamasters.com/masters-degrees/msc-degrees/?PG='
    prefix = '/masters-degrees/course/'
    exclude = ['\nMore details \n', '\nRead more \n', '\xa0Video(s)', '\xa0Student Profile(s)']

    # Create a list to store the URLs of the masters
    master_urls = []

    # Loop through the first 400 pages
    for page_number in range(1, 401):
        print(url + str(page_number))
        response = requests.get(url + str(page_number))

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")

            # Use BeautifulSoup to extract the URLs and append them to the master_urls list
            for link in soup.find_all('a', {'class':'courseLink'}):
                if link['href'][:len(prefix)] == prefix and not link.text in exclude:
                    master_urls.append((link['href'], link.text))


    # Save the collected URLs in a text file
    with open("master_urls.txt", "a") as file:
        for url in master_urls:
            file.write(url[0] + "\n")

#### 1.2. Crawl master's degree pages

In [5]:
flag = False

if flag:
    # Open the file and read its content
    with open("master_urls.txt", "r") as file:
        master_urls = [line.strip() for line in file.readlines()]

    # Create a directory to store HTML pages
    output_root_directory = "html_pages"
    os.makedirs(output_root_directory, exist_ok=True)  # create the directory if it doen't exist

    # Read 15 URLs at a time and create HTML pages
    subset_size = 15
    for i in range(0, len(master_urls), subset_size):
        subset = master_urls[i:i + subset_size] # extract 15 more urls

        # Create a subfolder for each page
        output_directory = os.path.join(output_root_directory, f"page_{i // subset_size + 1}")
        os.makedirs(output_directory, exist_ok=True)

        # Create an HTML page for each URL in the subset
        for url in subset:
            prefix = 'https://www.findamasters.com/'
            response = requests.get(prefix + url)  # sends a GET request

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, "html.parser")
                page_content = soup.prettify()  # extract the content of the page
                master_name = url.split("/")[-2]  # extract the name of the master from the URL

                # Check if the file already exists, and append a number if necessary
                page_filename = f"{output_directory}/{master_name}.html"
                counter = 1
                while os.path.exists(page_filename):
                    page_filename = f"{output_directory}/{master_name}({counter}).html"
                    counter += 1

                # Save the content in a HTML file
                with open(page_filename, "w", encoding="utf-8") as file:
                    file.write(page_content)
            time.sleep(1.5)

        print(f"page {i // subset_size + 1} completed")


#### 1.3 Parse downloaded pages

In [6]:
##MY IDEA --> I have to create a function that retrieve the required information from a html page. in order to do so I exploit the structure of the file.
#I use "try: except:" to avoid errors in the case that some information is missing

def extract_msc_page(msc_page_url):


    contents ={}
    # Parse HTML content
    page_soup = BeautifulSoup(msc_page_url, 'html.parser')

    #url
    try:
        canonical_link = page_soup.find('link', {'rel': 'canonical'})
        contents['url'] = canonical_link.get('href')
    except AttributeError:
        contents['url'] = ""


    # Course name
    try:
        page_links = page_soup.find('h1', {'class': 'text-white course-header__course-title'})
        name = page_links.get_text()
        contents["courseName"] = name.strip()
    except AttributeError:
        contents['courseName'] = ""

    # University name
    try:
        page_links = page_soup.find_all('a', {'class': 'course-header__institution'})
        contents['universityName'] = page_links[0].contents[0].strip()
    except (AttributeError, IndexError):
        contents['universityName'] = ""

    # Faculty name
    try:
        page_links = page_soup.find_all('a', {'class': 'course-header__department'})
        contents['facultyName'] = page_links[0].contents[0].strip()
    except (AttributeError, IndexError):
        contents['facultyName'] = ""

    # Full time
    try:
        page_links = page_soup.find('a', {'class': 'inheritFont concealLink text-decoration-none text-gray-600'})
        time = page_links.get_text().strip()
        contents['isItFullTime'] = time
    except AttributeError:
        contents['isItFullTime'] = ""

    # Description
    try:
        page_links = page_soup.find('div', {'id': 'Snippet'})
        description = page_links.get_text().strip()
        contents["description"] = description
    except AttributeError:
        contents['description'] = ""

    # Starting date
    try:
        page_links = page_soup.find('span', {'class': 'key-info__content key-info__start-date py-2 pr-md-3 text-nowrap d-block d-md-inline-block'})
        starting = page_links.get_text().strip()
        contents["startDate"] = starting
    except AttributeError:
        contents['startDate'] = ""

    # Fees
    try:
        page_links = page_soup.find('div', {'class': 'course-sections course-sections__fees tight col-xs-24'})
        fees = page_links.get_text().strip()
        contents["fees"] = fees
    except AttributeError:
        contents['fees'] = ""

    # Modality
    try:
        page_links = page_soup.find('a', {'title': 'View all MSc courses'})
        modality = page_links.get_text().strip()
        contents["modality"] = modality
    except AttributeError:
        contents['modality'] = ""

    # Duration
    try:
        page_links = page_soup.find('span', {'class': 'key-info__content key-info__duration py-2 pr-md-3 d-block d-md-inline-block'})
        duration = page_links.get_text().strip()
        contents["duration"] = duration
    except AttributeError:
        contents['duration'] = ""

    # City
    try:
        page_links = page_soup.find('a', {'class': 'card-badge text-wrap text-left badge badge-gray-200 p-2 m-1 font-weight-light course-data course-data__city'})
        city = page_links.get_text().strip()
        contents["city"] = city
    except AttributeError:
        contents['city'] = ""

    # Country
    try:
        page_links = page_soup.find('a', {'class': 'card-badge text-wrap text-left badge badge-gray-200 p-2 m-1 font-weight-light course-data course-data__country'})
        country = page_links.get_text().strip()
        contents["country"] = country
    except AttributeError:
        contents['country'] = ""

    # Administration
    try:
        page_links = page_soup.find('a', {'class': 'card-badge text-wrap text-left badge badge-gray-200 p-2 m-1 font-weight-light course-data course-data__on-campus'})
        administration = page_links.get_text().strip()
        contents["administration"] = administration
    except AttributeError:
        contents["administration"] = ""


    return contents


In [7]:
## MY IDEA --> I apply the function above for all the files. I am applying it for all files in a folder for all folders contained into a folder

flag = False

if flag:
    folder_path = 'html_pages/'

    result_df = pd.DataFrame()
    # Walk through the directory and its subdirectories
    for foldername, subfolders, filenames in os.walk(folder_path):
        # Iterate through all the files in the current subdirectory
        for filename in filenames:
            # Construct the file path
            path = os.path.join(foldername, filename)

            # Check if the file exists before attempting to open it
            if os.path.exists(path):
                # Print the file path
                print(path)

                # Open the file in read mode with UTF-8 encoding
                with open(path, 'r', encoding='utf-8') as file:
                    # Read the HTML content of the file
                    html_content = file.read()

                    # Call the function to extract information from the HTML content
                    result_dict = extract_msc_page(html_content)
                    result_df = result_df.append(result_dict, ignore_index=True)


            else:
                # Print a message if the file is not found
                print(f"File not found: {path}")


    #clean an imperfection in the fees section
    result_df['fees'] = result_df['fees'].str.replace('Fees', '')

    #save it into a file to store it. so that I do not have to run it again
    result_df.to_json('html_pages.json', orient='records', lines=True)


In [8]:
# Create ths tsv files

flag = False

if flag:
    data = pd.read_json("html_pages.json", lines=True)

    output_directory_masters = "tsv_files"
    os.makedirs(output_directory_masters, exist_ok=True)

    # Extract relevant columns
    selected_columns = ['courseName', 'universityName', 'facultyName', 'isItFullTime', 'description', 'startDate', 'fees', 'modality', 'duration', 'city', 'country', 'administration', 'url']

    # Concatenate all rows into a single DataFrame
    data_courses = data[selected_columns].replace('\n', '', regex=True).replace('\t', '', regex=True) # replace \t and \n within column values to preserve the tsv structure

    # Convert the concatenated data to a string with '\n' as the separator
    all_courses_string = data_courses.to_csv(sep='\t', index=False, header=False, lineterminator='\n')

    # Split the string into individual lines (using \n as separatoor) to create singles tsv file
    lines = all_courses_string.split('\n')

    for index, line in enumerate(lines):
        # Skip empty lines
        if not line:
            continue

        # Create a unique TSV file for each line
        file_name = f"course_{index}.tsv"
        file_path = os.path.join(output_directory_masters, file_name)

        # Write the line and \n to the TSV file
        with open(file_path, 'w') as file:
            file.write(line + '\n')


### **2. Search Engine**

##### 2.0.0) Preprocessing the text

In [9]:
dataset = pd.read_json("html_pages.json", lines=True)  # read the dataset from the created json file
dataset = dataset[dataset.description != '']  # filter rows where the 'description' is empty

# STEMMING
stemmer = PorterStemmer()  # create an instance of Porter Stemmer
dataset['preprocessed_description'] = dataset.description.apply(lambda row: [stemmer.stem(word) for word in row.split(' ')])  # reduce words of description column to their root form and create a new column to store the result

# REMOVE STOPWORDS
nltk.download('stopwords')
list_stopwords = stopwords.words('english')  # retrieves the English stopwords from the nltk stopwords dataset
dataset['preprocessed_description'] = dataset.description.apply(lambda row: [stemmer.stem(word) for word in row.split(' ') if not word in list_stopwords])  # now 'descr_clean' column contains lists of cleaned and stemmed words

# REMOVE PUNCTUATION
nltk.download('punkt')
nltk.download('wordnet')
dataset['preprocessed_description'] = dataset.description.apply(lambda row: [stemmer.stem(word) for word in nltk.word_tokenize(row) if not word in list_stopwords and word.isalnum()]) # now 'descr_clean' column contains lists of cleaned and stemmed words without punctuation

# Join the lists of words into a single string and create a new column
dataset['preprocessed_description_str'] = dataset.preprocessed_description.apply(lambda row: ' '.join(row))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


#### 2.0.1) Preprocessing the fees column

In [10]:
from forex_python.converter import CurrencyRates

pattern1 = r'([$£¥€]|USD|GBP|EUR|euro|euros|JPY|HK$|HK|HKD$|HKD|ISK|SEK)\s*([\d,]+(?:\.\d+)?)'  # regex to match pattern "symbol number"
pattern2 = r'([\d,]+(?:\.\d+)?)\s*([$£¥€]|USD|GBP|EUR|euro|euros|JPY|HK$|HK|HKD$|HKD|ISK|SEK)'  # regex to match pattern "number symbol"

currency_rates = CurrencyRates()  # create a CurrencyRates instance


def extract_and_convert(row, common_currency="USD"):  # method to extract fee values from the 'fees' column and convert them all to the same currency (Dollars)

    matches1 = re.finditer(pattern1, row)
    matches2 = re.finditer(pattern2, row)

    # initialize 'max_value' and the currency of the maximum value to update within the for loops below
    max_value = 0
    max_currency = None
    max_match = None

    # iterate over the matches obtained from the first regex
    for match in matches1:
        currency_symbol = match.group(1)
        numeric_part = match.group(2)


        # Try to convert numeric part to float
        try:
            numeric_value = float(numeric_part.replace(',', '').replace('.', '')) # remove the commas and dots used as thousands separators and cast to float
        except ValueError:
            continue

        # update max value if necessary
        if numeric_value > max_value:
            max_value = numeric_value
            max_currency = currency_symbol
            max_match = match

    # iterate over the matches obtained from the second regex and do what was done in the above loop
    for match in matches2:
        currency_symbol = match.group(2)
        numeric_part = match.group(1)

        try:
            numeric_value = float(numeric_part.replace(',', '').replace('.', ''))
        except ValueError:
            continue

        if numeric_value > max_value:
            max_value = numeric_value
            max_currency = currency_symbol
            max_match = match


    if max_match:
        # Convert the price to the common currency
        converted_price = convert_currency(max_currency, max_value, common_currency)  # call the convert_currency method defined below
        return converted_price
    else:
        return None



# Define a function to convert currency
def convert_currency(currency_symbol, amount, target_currency):
    try:
        # Map currency symbols to ISO codes
        currency_code = symbol_to_code(currency_symbol)

        if currency_code is not None:
            # Use the forex-python library to get the exchange rate and perform the conversion
            exchange_rate = currency_rates.get_rate(currency_code, target_currency)
            converted_amount = round(amount * exchange_rate, 2)
            print(f"currency exchange: {amount:.2f}{currency_code} --> {converted_amount:.2f}{target_currency}")
            return converted_amount
        else:
            print(f"Unsupported currency symbol: {currency_symbol}")
            return None
    except Exception as e:
        print(f"Error converting currency: {e}")
        return None



# Define a function to map currency symbols to ISO codes using a dictionary
def symbol_to_code(symbol):
    symbol_map = {
        '$': 'USD',
        '£': 'GBP',
        '¥': 'JPY',
        '€': 'EUR',
        'USD': 'USD',
        'GBP': 'GBP',
        'EUR': 'EUR',
        'euro': 'EUR',
        'euros': 'EUR',
        'JPY': 'JPY',
        'HKD$': 'HKD',
        'HKD': 'HKD',
        'HK$': 'HKD',
        'HK': 'HKD',
        'ISK': 'ISK',
        'SEK': 'SEK'
    }
    return symbol_map.get(symbol, None)


# Apply the function to the 'fees' column and create a new 'fees(USD)' column
dataset['fees(USD)'] = dataset['fees'].apply(lambda x: extract_and_convert(x, common_currency='USD'))


currency exchange: 34750.00GBP --> 42542.88USD
currency exchange: 31000.00GBP --> 37951.92USD
currency exchange: 18000.00EUR --> 19206.00USD
currency exchange: 18000.00EUR --> 19206.00USD
currency exchange: 15000.00EUR --> 16005.00USD
currency exchange: 15000.00EUR --> 16005.00USD
currency exchange: 28750.00GBP --> 35197.35USD
currency exchange: 722.00GBP --> 883.91USD
currency exchange: 31000.00GBP --> 37951.92USD
currency exchange: 28000.00GBP --> 34279.16USD
currency exchange: 10500.00GBP --> 12854.68USD
currency exchange: 22600.00EUR --> 24114.20USD
currency exchange: 22600.00EUR --> 24114.20USD
currency exchange: 4000.00EUR --> 4268.00USD
currency exchange: 18000.00GBP --> 22036.60USD
currency exchange: 280600.00SEK --> 25788.13USD
currency exchange: 2500.00EUR --> 2667.50USD
currency exchange: 26350.00GBP --> 32259.14USD
currency exchange: 26350.00GBP --> 32259.14USD
currency exchange: 26350.00GBP --> 32259.14USD
currency exchange: 26350.00GBP --> 32259.14USD
currency exchange: 4

### 2.1. Conjunctive query

#### 2.1.1) Create your index!

In [11]:
#step 1:
#Create a file named vocabulary, in the format you prefer, that maps each word to an integer (term_id).

vocabulary = set()
dataset.preprocessed_description.apply(lambda row: [vocabulary.add(word) for word in row])


0       [None, None, None, None, None, None, None, Non...
1       [None, None, None, None, None, None, None, Non...
2       [None, None, None, None, None, None, None, Non...
3       [None, None, None, None, None, None, None, Non...
4       [None, None, None, None, None, None, None, Non...
                              ...                        
5995    [None, None, None, None, None, None, None, Non...
5996    [None, None, None, None, None, None, None, Non...
5997    [None, None, None, None, None, None, None, Non...
5998    [None, None, None, None, None, None, None, Non...
5999    [None, None, None, None, None, None, None, Non...
Name: preprocessed_description, Length: 5979, dtype: object

In [12]:
vocabulary_alt = Counter(reduce(lambda x, y: x + y, dataset.preprocessed_description.values)).keys()

index = {}
unique_id = 1
for word in list(vocabulary_alt):
  index[unique_id] = word
  unique_id += 1

# Create a DataFrame from the vocabulary
vocabulary_df_alt = pd.DataFrame(list(index.items()), columns=['term_id', 'word'])

# Save the vocabulary DataFrame to a CSV file
vocabulary_df_alt.to_csv('vocabulary_alt.csv', index=False)

terms = pd.DataFrame(data=list(vocabulary_alt), columns=['term'])


#### 2.1.2) Execute the query


In [13]:
def select_docs(query):

    # Split the personalized query into individual words
    query_words = query.split()

    # Your modified code to get the indices of documents containing all words in the personalized query
    selected_indices = list(dataset.loc[dataset.description.apply(lambda row: all(word in row for word in query_words))].index)

    # Extract information for each selected document
    selected_documents = dataset.loc[selected_indices, ['courseName', 'universityName', 'description', 'url']]

    # Print the information for each selected document
    return selected_documents


In [14]:
select_docs("advanced knowledge")  # try with user query "advanced knowledge"

,courseName,universityName,description,url
2,Accounting and Finance - MSc,University of Leeds,Businesses and governments rely on sound finan...,https://www.findamasters.com/masters-degrees/c...
12,Analytical Toxicology MSc,King’s College London,The Analytical Toxicology MSc is a unique stud...,https://www.findamasters.com/masters-degrees/c...
28,"Healthcare Technologies MSc, MRes",King’s College London,The Healthcare Technologies MSc/MRes will trai...,https://www.findamasters.com/masters-degrees/c...
36,Biomaterials and Tissue Engineering MSc,University College London,Register your interest in graduate study at UC...,https://www.findamasters.com/masters-degrees/c...
37,Biomedical and Analytical Science MSc,University of Huddersfield,The Biomedical and Analytical Science MSc cour...,https://www.findamasters.com/masters-degrees/c...
...,...,...,...,...
5879,Automotive Technology (MSc),University of Bath,Study a master’s that focuses on giving you th...,https://www.findamasters.com/masters-degrees/c...
5910,Banking and Finance MSc,Brunel University London,Ask Brunel\n \n\n\n\n ...,https://www.findamasters.com/masters-degrees/c...
5924,Banking Innovation and Risk Analytics MSc,University of Edinburgh,Programme description\n \n\n\n ...,https://www.findamasters.com/masters-degrees/c...
5951,Big Data Technologies MSc,University of East London,This programme is ideal if you want to pursue ...,https://www.findamasters.com/masters-degrees/c...


### 2.2) Conjunctive query & Ranking score

#### 2.2.1) Inverted index

In [15]:
#max_df is set to 0.5, indicating that terms with a document frequency higher than 50% will be ignored
#min_df is set to 2, indicating that terms with a document frequency lower than 2 will be ignored

# Convert the tokenized lists to strings
dataset['preprocessed_description_str'] = dataset['preprocessed_description'].apply(lambda x: ' '.join(x))

# Use the default tokenizer provided by TfidfVectorizer
tfidf = TfidfVectorizer(max_df=0.5, min_df=2)
results = tfidf.fit_transform(dataset.preprocessed_description_str)
result_dense = results.todense()

tfidf_data = pd.DataFrame(result_dense.tolist(), index=dataset.index, columns=tfidf.get_feature_names_out())


In [16]:
def search_engine(query, k):

    # preprocessing of the input query
    query = ' '.join([stemmer.stem(word) for word in nltk.word_tokenize(query) if not word in list_stopwords and word.isalnum()])

    query_tfidf = tfidf.transform([query])

    # Compute cosine similarity between the query and all documents
    query_similarity = cosine_similarity(query_tfidf, tfidf_data)


    # Get indices of the top-k documents using a heap
    heap = []
    for i, sim_score in enumerate(query_similarity[0]):
        if len(heap) < k:
            heapq.heappush(heap, (sim_score, i))
        else:
            if sim_score > heap[0][0]:
                heapq.heappop(heap)
                heapq.heappush(heap, (sim_score, i))

    # Retrieve the top-k documents from the heap
    top_k_documents = []
    while heap:
        sim_score, index = heapq.heappop(heap)
        document_info = {
            'courseName': dataset.loc[index, 'courseName'],
            'universityName': dataset.loc[index, 'universityName'],
            'description': dataset.loc[index, 'description'],
            'url': dataset.loc[index, 'url'],
            'Similarity Score': sim_score
        }
        top_k_documents.append(document_info)

        result_df = pd.DataFrame(top_k_documents)

    # Sort the result by similarity score in ascending order
    top_k_documents = sorted(top_k_documents, key=lambda x: x['Similarity Score'], reverse=True)
    result_df = pd.DataFrame(top_k_documents)

    return result_df

##### 2.2.2) Execute the query

In [17]:
# The top_k_results DataFrame will contain information about the top-k documents that are most similar to the query, with the most similar document listed first.
query = str(input()) # read the user's query
top_k_results = search_engine(query, k=5)

top_k_results

sapienza


,courseName,universityName,description,url,Similarity Score
0,Ethical Hacking and Cyber Security - MSc/PGDip,Abertay University,Fully certified by the National Cybersecurity ...,https://www.findamasters.com/masters-degrees/c...,0.093309
1,Euro-Mediterranean Master of Neuroscience (EMN...,University of Bordeaux,EMN-Online is a 2 year Master programme in Neu...,https://www.findamasters.com/masters-degrees/c...,0.093309
2,Applied Psychology and Economic Behaviour,University of Bath,Explore the intersection between psychology an...,https://www.findamasters.com/masters-degrees/c...,0.093309
3,"Accounting, Financial Management and Digital B...",University of Reading,Embark on a professional accounting career wit...,https://www.findamasters.com/masters-degrees/c...,0.000000
4,Addictions MSc,King’s College London,Join us for an online session for prospective ...,https://www.findamasters.com/masters-degrees/c...,0.000000


### **3. Define a new score!**

In [22]:
### THE IDEA --> given a text query, I exploit the previous search engine. I compute the cosine similarity score and
#then I adjust it taking into consideration also the other variable of the dataset as it follows:
# if it has a fee(so different than nan), give it credit
# if "administration" is not empty, give it credit
# if it is full time, give it credit
#after doing it, I took the top documents. then I compare the results

def new_search_engine(query, k):

    # preprocessing of the input query
    query = ' '.join([stemmer.stem(word) for word in nltk.word_tokenize(query) if not word in list_stopwords and word.isalnum()])

    query_tfidf = tfidf.transform([query])

    # Compute cosine similarity between the query and all documents
    query_similarity = cosine_similarity(query_tfidf, tfidf_data)

    # Get indices of the top-k documents using a heap
    heap = []
    for i, sim_score in enumerate(query_similarity[0]):
        if len(heap) < k:
            heapq.heappush(heap, (sim_score, i))
        else:
            if sim_score > heap[0][0]:
                heapq.heappop(heap)
                heapq.heappush(heap, (sim_score, i))

    # Retrieve the top-k documents from the heap
    top_k_documents = []
    while heap:
        sim_score, index = heapq.heappop(heap)

        # Additional conditions to adjust the similarity score
        if pd.notna(dataset.loc[index, 'fees(USD)']):
            sim_score += 0.2  # Give credit for having fee value

        if dataset.loc[index, 'administration'] != '':
            sim_score += 0.2  # Give credit for non-empty administration field

        if dataset.loc[index, 'isItFullTime'] == 'Full time':
            sim_score += 0.2  # Give credit for being full time (we want to to give proirity to full-time courses)

        document_info = {
            'courseName': dataset.loc[index, 'courseName'],
            'universityName': dataset.loc[index, 'universityName'],
            'description': dataset.loc[index, 'description'],
            'url': dataset.loc[index, 'url'],
            'Similarity Score': sim_score
        }
        top_k_documents.append(document_info)

    # Sort the result by similarity score in ascending order
    top_k_documents = sorted(top_k_documents, key=lambda x: x['Similarity Score'], reverse=True)
    result_df = pd.DataFrame(top_k_documents)

    return result_df


In [65]:
personalized_query = str(input())
top_k_results = new_search_engine(personalized_query, k=10)

top_k_results

Sapienza


,courseName,universityName,description,url,Similarity Score
0,Ethical Hacking and Cyber Security - MSc/PGDip,Abertay University,Fully certified by the National Cybersecurity ...,https://www.findamasters.com/masters-degrees/c...,0.693309
1,Advanced Chemical Engineering - MSc,University of Leeds,The Advanced Chemical Engineering MSc at Leeds...,https://www.findamasters.com/masters-degrees/c...,0.600000
2,Advanced Master in Financial Markets,Solvay Brussels School,Programme overview\n \n\n ...,https://www.findamasters.com/masters-degrees/c...,0.600000
3,Advanced Master in Innovation & Strategic Mana...,Solvay Brussels School,Programme overview\n \n\n ...,https://www.findamasters.com/masters-degrees/c...,0.600000
4,"Agricultural, Environmental and Resource Econo...",University of Helsinki,Goal of the pro­gramme\n \n\n ...,https://www.findamasters.com/masters-degrees/c...,0.600000
5,Applied Psychology and Economic Behaviour,University of Bath,Explore the intersection between psychology an...,https://www.findamasters.com/masters-degrees/c...,0.493309
6,"Accounting, Financial Management and Digital B...",University of Reading,Embark on a professional accounting career wit...,https://www.findamasters.com/masters-degrees/c...,0.400000
7,Addictions MSc,King’s College London,Join us for an online session for prospective ...,https://www.findamasters.com/masters-degrees/c...,0.400000
8,Advanced Physiotherapy Practice - MSc,Glasgow Caledonian University,Progress your career as a physiotherapist with...,https://www.findamasters.com/masters-degrees/c...,0.400000
9,Euro-Mediterranean Master of Neuroscience (EMN...,University of Bordeaux,EMN-Online is a 2 year Master programme in Neu...,https://www.findamasters.com/masters-degrees/c...,0.293309


**4. VISUALIZATION**

In [66]:
#Get the columns of the original dataframe we are interested in for the maps
cities = dataset[["courseName",	"universityName",	"city", "country", "fees(USD)"]]

#Get the information for our top_k_results
cities_top_k = pd.merge(cities, top_k_results, on=["courseName", "universityName"], how='right')
cities_top_k

,courseName,universityName,city,country,fees(USD),description,url,Similarity Score
0,Ethical Hacking and Cyber Security - MSc/PGDip,Abertay University,Dundee,United Kingdom,20812.35,Fully certified by the National Cybersecurity ...,https://www.findamasters.com/masters-degrees/c...,0.693309
1,Advanced Chemical Engineering - MSc,University of Leeds,Leeds,United Kingdom,37951.92,The Advanced Chemical Engineering MSc at Leeds...,https://www.findamasters.com/masters-degrees/c...,0.600000
2,Advanced Master in Financial Markets,Solvay Brussels School,Brussels,Belgium,19206.00,Programme overview\n \n\n ...,https://www.findamasters.com/masters-degrees/c...,0.600000
3,Advanced Master in Innovation & Strategic Mana...,Solvay Brussels School,Brussels,Belgium,19206.00,Programme overview\n \n\n ...,https://www.findamasters.com/masters-degrees/c...,0.600000
4,"Agricultural, Environmental and Resource Econo...",University of Helsinki,Helsinki,Finland,16005.00,Goal of the pro­gramme\n \n\n ...,https://www.findamasters.com/masters-degrees/c...,0.600000
5,Applied Psychology and Economic Behaviour,University of Bath,Bath,United Kingdom,NaN,Explore the intersection between psychology an...,https://www.findamasters.com/masters-degrees/c...,0.493309
6,"Accounting, Financial Management and Digital B...",University of Reading,Reading,United Kingdom,NaN,Embark on a professional accounting career wit...,https://www.findamasters.com/masters-degrees/c...,0.400000
7,Addictions MSc,King’s College London,London,United Kingdom,NaN,Join us for an online session for prospective ...,https://www.findamasters.com/masters-degrees/c...,0.400000
8,Advanced Physiotherapy Practice - MSc,Glasgow Caledonian University,Glasgow,United Kingdom,NaN,Progress your career as a physiotherapist with...,https://www.findamasters.com/masters-degrees/c...,0.400000
9,Euro-Mediterranean Master of Neuroscience (EMN...,University of Bordeaux,Bordeaux,France,NaN,EMN-Online is a 2 year Master programme in Neu...,https://www.findamasters.com/masters-degrees/c...,0.293309


In [75]:
# Source: https://www.tutorialspoint.com/how-to-get-the-longitude-and-latitude-of-a-city-using-python
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Trial") #the user_agent can be any string, i chose "Trial".
latitudes_list = []
longitudes_list = []
for city_name in cities_top_k["city"]:
  #in case we can't find coordinates for any of the cities, we use try and except blocks.
  try:
    location = geolocator.geocode(city_name)
    #if location information is available, append the lists accordingly.
    if location:
      latitudes_list.append(location.latitude)
      longitudes_list.append(location.longitude)
    #if the location information is not avaiable, append the lists with NaN.
    else:
      latitudes_list.append(np.nan)
      longitudes_list.append(np.nan)

  except Exception as e:
      print(f"Error occurred while geocoding {city_name}: {e}")
      latitudes_list.append(np.nan)
      longitudes_list.append(np.nan)

In [76]:
#Add the coordinate columns into our dataframe.
cities_top_k["latitude"] =latitudes_list
cities_top_k["longitude"] = longitudes_list
cities_top_k

,courseName,universityName,city,country,fees(USD),description,url,Similarity Score,latitude,longitude
0,Ethical Hacking and Cyber Security - MSc/PGDip,Abertay University,Dundee,United Kingdom,20812.35,Fully certified by the National Cybersecurity ...,https://www.findamasters.com/masters-degrees/c...,0.693309,56.460594,-2.970190
1,Advanced Chemical Engineering - MSc,University of Leeds,Leeds,United Kingdom,37951.92,The Advanced Chemical Engineering MSc at Leeds...,https://www.findamasters.com/masters-degrees/c...,0.600000,53.797418,-1.543794
2,Advanced Master in Financial Markets,Solvay Brussels School,Brussels,Belgium,19206.00,Programme overview\n \n\n ...,https://www.findamasters.com/masters-degrees/c...,0.600000,50.855002,4.351233
3,Advanced Master in Innovation & Strategic Mana...,Solvay Brussels School,Brussels,Belgium,19206.00,Programme overview\n \n\n ...,https://www.findamasters.com/masters-degrees/c...,0.600000,50.855002,4.351233
4,"Agricultural, Environmental and Resource Econo...",University of Helsinki,Helsinki,Finland,16005.00,Goal of the pro­gramme\n \n\n ...,https://www.findamasters.com/masters-degrees/c...,0.600000,60.167488,24.942747
5,Applied Psychology and Economic Behaviour,University of Bath,Bath,United Kingdom,NaN,Explore the intersection between psychology an...,https://www.findamasters.com/masters-degrees/c...,0.493309,51.381386,-2.359696
6,"Accounting, Financial Management and Digital B...",University of Reading,Reading,United Kingdom,NaN,Embark on a professional accounting career wit...,https://www.findamasters.com/masters-degrees/c...,0.400000,51.456424,-0.970066
7,Addictions MSc,King’s College London,London,United Kingdom,NaN,Join us for an online session for prospective ...,https://www.findamasters.com/masters-degrees/c...,0.400000,51.489334,-0.144055
8,Advanced Physiotherapy Practice - MSc,Glasgow Caledonian University,Glasgow,United Kingdom,NaN,Progress your career as a physiotherapist with...,https://www.findamasters.com/masters-degrees/c...,0.400000,55.861155,-4.250169
9,Euro-Mediterranean Master of Neuroscience (EMN...,University of Bordeaux,Bordeaux,France,NaN,EMN-Online is a 2 year Master programme in Neu...,https://www.findamasters.com/masters-degrees/c...,0.293309,44.841225,-0.580036


In [27]:
import folium

In [91]:
# Initialize a folium map
m = folium.Map(location=[cities_top_k["latitude"].mean(), cities_top_k["longitude"].mean()], zoom_start=5)

for i, row in cities_top_k.iterrows():
  #I got help from ChatGPT to show the pop-up in a tabular form.
    popup_text = f"""
    <table style="width:100%">
      <tr>
        <th>University</th>
        <td>{row['universityName']}</td>
      </tr>
      <tr>
        <th>City</th>
        <td>{row['city']}</td>
      </tr>
      <tr>
        <th>Country</th>
        <td>{row['country']}</td>
      </tr>
      <tr>
        <th>Fees</th>
        <td>{row['fees(USD)']}</td>
      </tr>
    </table>
    """
    # Add the map points
    folium.CircleMarker(location=[row["latitude"], row["longitude"]],
                      radius=15,
                      popup=popup_text,
                      fill=True,
                      fill_color="red",
                      fill_opacity=0.6,
                      color="red").add_to(m)

    # Add the labels to map points
    folium.Marker(location=[row["latitude"], row["longitude"]],
                  icon=None,
                  popup=folium.Popup(popup_text, max_width=300),
                  tooltip=row['universityName'],
                  ).add_to(m)
m

The next part will be for an alternative graph.
In the following graph, we are aiming to emphasize the fee information.

In [47]:
import plotly.express as px

In [79]:
cities_top_k = cities_top_k.sort_values(by="fees(USD)", ascending=True)

In [87]:
#In order to create a map with different radius values for different fees, we have to "normalize" the fees. To do that, we use percentages.
#But in order to draw more attention to the program with the least fee, we wanted to make the radius bigger if the fee is smaller.
#However, since we are also dealing with NANs in the fees column, our code assigns a spesific radius value (10) to NAN fees.
#But since the colours of the data points are also determined according to the fee values, the viewer can conclude which courses don't have the fee information by looking at the colours.
#The lighter the colour, the lower the fee is.

fee_normalizer_df = pd.DataFrame()
fee_normalizer_df["unique_fees"] = cities_top_k["fees(USD)"].unique()
my_list = []
#if more than 1 course in my dataframe have fee information, I can calculate a percentage.
if fee_normalizer_df['unique_fees'].count()>1:
  # Check if there are any NAN values
  if fee_normalizer_df['unique_fees'].isna().any():
      for i in range(len(fee_normalizer_df)):
          # Check if the specific row value in 'unique_fees' is NaN
          if pd.isnull(fee_normalizer_df["unique_fees"].iloc[i]):
              #for NaN values in my dataframe, I will assign a specific value (10) for the radius.
              my_list.append(10)
          #I will firstly calculate the percentage in the fee sum and then append my list with (100-percantage) since I want to make the courses with less fees more visible in the graph.
          else:
              percentage = (fee_normalizer_df["unique_fees"].iloc[i] / fee_normalizer_df["unique_fees"].sum()) * 100
              my_list.append(100 - percentage)
  #if there is no NAN values, calculate with the same logic above for the whole fee information.
  else:
      my_list = [100 - ((x / fee_normalizer_df["unique_fees"].sum()) * 100) for x in fee_normalizer_df["unique_fees"]]

  fee_normalizer_df["reverse_percentage"] = my_list
#if there is only 1 course with fee information or no course at all, assign a specific value (10) for the radius.
else:
  fee_normalizer_df["reverse_percentage"] = 10

# Merge the DataFrames based on the 'fees(USD)' and 'unique_fees' columns
final_df = pd.merge(cities_top_k, fee_normalizer_df, left_on='fees(USD)', right_on='unique_fees', how='inner')

final_df

,courseName,universityName,city,country,fees(USD),description,url,Similarity Score,latitude,longitude,unique_fees,reverse_percentage
0,"Agricultural, Environmental and Resource Econo...",University of Helsinki,Helsinki,Finland,16005.00,Goal of the pro­gramme\n \n\n ...,https://www.findamasters.com/masters-degrees/c...,0.600000,60.167488,24.942747,16005.00,82.968924
1,Advanced Master in Financial Markets,Solvay Brussels School,Brussels,Belgium,19206.00,Programme overview\n \n\n ...,https://www.findamasters.com/masters-degrees/c...,0.600000,50.855002,4.351233,19206.00,79.562708
2,Advanced Master in Innovation & Strategic Mana...,Solvay Brussels School,Brussels,Belgium,19206.00,Programme overview\n \n\n ...,https://www.findamasters.com/masters-degrees/c...,0.600000,50.855002,4.351233,19206.00,79.562708
3,Ethical Hacking and Cyber Security - MSc/PGDip,Abertay University,Dundee,United Kingdom,20812.35,Fully certified by the National Cybersecurity ...,https://www.findamasters.com/masters-degrees/c...,0.693309,56.460594,-2.970190,20812.35,77.853376
4,Advanced Chemical Engineering - MSc,University of Leeds,Leeds,United Kingdom,37951.92,The Advanced Chemical Engineering MSc at Leeds...,https://www.findamasters.com/masters-degrees/c...,0.600000,53.797418,-1.543794,37951.92,59.614992
5,Applied Psychology and Economic Behaviour,University of Bath,Bath,United Kingdom,NaN,Explore the intersection between psychology an...,https://www.findamasters.com/masters-degrees/c...,0.493309,51.381386,-2.359696,NaN,10.000000
6,"Accounting, Financial Management and Digital B...",University of Reading,Reading,United Kingdom,NaN,Embark on a professional accounting career wit...,https://www.findamasters.com/masters-degrees/c...,0.400000,51.456424,-0.970066,NaN,10.000000
7,Addictions MSc,King’s College London,London,United Kingdom,NaN,Join us for an online session for prospective ...,https://www.findamasters.com/masters-degrees/c...,0.400000,51.489334,-0.144055,NaN,10.000000
8,Advanced Physiotherapy Practice - MSc,Glasgow Caledonian University,Glasgow,United Kingdom,NaN,Progress your career as a physiotherapist with...,https://www.findamasters.com/masters-degrees/c...,0.400000,55.861155,-4.250169,NaN,10.000000
9,Euro-Mediterranean Master of Neuroscience (EMN...,University of Bordeaux,Bordeaux,France,NaN,EMN-Online is a 2 year Master programme in Neu...,https://www.findamasters.com/masters-degrees/c...,0.293309,44.841225,-0.580036,NaN,10.000000


In [88]:
fig = px.scatter_geo(final_df, lat = "latitude", lon="longitude",
                     color="fees(USD)",
                     hover_name="city", size="reverse_percentage",
                     projection="natural earth",
                     color_continuous_scale='Greens_r',
                     range_color=(10, final_df['fees(USD)'].max()))
fig.show()


### **6. COMMAND LINE**


![commandline.bmp](attachment:commandline.bmp)

### **7. ALGORITMIC QUESTION**

**1. Implement a code to solve the above mentioned problem.**

In [94]:
first_line = list(map(int, input().split()))
d = first_line[0]
sumHours = first_line[1]

2 5


TypeError: ignored

In [ ]:
min_times_list = []
max_times_list = []

error_flag = False #to ensure the input is in a logically correct format
for i in range(d):
  input_lines = list(map(int, input().split()))
  if input_lines[0] > input_lines[1]:
    print ("Minimum Time Cannot Exceed Maximum Time")
    error_flag = True
    break
  else:
    min_times_list.append(input_lines[0])
    max_times_list.append(input_lines[1])

if not error_flag: #if the input is in a logically correct format, we save the values to our lists.
  min_times_list = min_times_list
  max_times_list = max_times_list
else: #if the input is not in a logically correct format, we do not save the input into our lists.
  min_times_list = []
  max_times_list = []

In [ ]:
worked_hours_list = []
if (sum(min_times_list)>sumHours) or (len(min_times_list)==0): # the case which leads to an infeasable solution.
  print ("NO")
elif sum(min_times_list)==sumHours: # this part is only to save time if the total hour can only be achieved by using the minimum hours.
  print ("YES")
  print (min_times_list)
else:
  print ("YES")
  while True: #to be able to loop until the following if condition is satisfied.
    for i in range (d-1): #generate the hours for d-1 days randomly.
      worked_hours_list.append(random.randint(min_times_list[i], max_times_list[i]))
    current_sum = sum(worked_hours_list)
    if ((sumHours - current_sum) >= min_times_list[d-1]) and ((sumHours - current_sum) <= max_times_list[d-1]): # logical condition to check if there is an answer with the random hours created so far.
      worked_hours_list.append(sumHours - current_sum) #the last day's hour is equal to the difference between the total hour and the sum of the other days.
      print (worked_hours_list)
      break
    else:
      worked_hours_list = []

**2. What is the time complexity (the Big O notation) of your solution? Please provide a detailed explanation of how you calculated the time complexity.**

In order to calculate the time complexity, I mainly focused on the loops. We have the first "for" loop (for i in range(d)) which has the time complexity of O(n). The statements inside this "for" loop is repetated d times. Time complexity of all our "if" statements are equal to O(1), since they will be executed once. Then, the most important part, we have a "for" loop inside a "while" loop. The "while" loop will be executed until an "if" statement is true. In the best case, it will be executed only once. But in the worst case, the inside of the "while" loop will be executed indefinite amount. This is because there is the randomness element. Considering these, the time complexity cannot be calculated.

**3. Ask ChatGPT or any other LLM chatbot tool to check your code's time complexity (the Big O notation). Compare your answer to theirs. Do you believe this is correct? If the two differ, which one is right? (why?)**


The answer of ChatGPT:

> "Input Reading and Validation: The code reads input values, including the number of days (d), total hours (sumHours), and minimum and maximum hours for each day. The loop from for i in range(d) takes O(d) time, as it iterates over the number of days. Within this loop, it checks the validity of input and populates min_times_list and max_times_list.

> Validation and Early Exit: There's a check (if not error_flag) to determine whether the input is valid. If it is, the code proceeds to the next steps. Otherwise, it exits early based on the input validation, which takes O(1) time.

> Handling Infeasible Solutions: There are checks to handle cases where the total minimum hours are greater than the provided total hours or if the minimum times list is empty. These checks are done in O(d) time, as they involve checking the entire list.

> Main Logic (While Loop): There's a while True loop, indicating an indefinite loop until a certain condition is met. This can potentially run multiple times, but the number of iterations is not fixed or determined solely by the input size. Inside this loop, there's a for loop iterating d - 1 times, generating random hours within certain ranges. This loop's time complexity is O(d) as it iterates over the number of days.

> Condition Check and Final Output: Within the while loop, there's a check (if ...) involving arithmetic operations and comparisons. This condition is checked repeatedly until it satisfies certain criteria. The exact number of iterations of this check isn't straightforward to determine without knowing the randomness of the generated values.

> Overall Time Complexity Analysis: The time complexity is mostly influenced by the loops that iterate based on the number of days (d). Input validation and handling infeasible solutions also contribute to the time complexity but in a linear manner. The while loop's time complexity, driven by randomness and the condition inside it, can vary significantly depending on the generated values, making it challenging to precisely determine the overall time complexity in terms of 'n' or 'd'. In conclusion, the time complexity of this code snippet is predominantly influenced by the iterations based on the number of days (d), but the overall complexity is challenging to express definitively due to the unpredictable nature of the loop's termination condition and the randomness involved in the generation of values."

My answer and this answer are almost the same. ChatGPT's answer has more explanations but we both agree the overall time complexity cannot be calculated because of the randomness.

**4. What do you think of the optimality of your code? Do you believe it is optimal? Can you improve? Please elaborate on your response.**

This depends on what Leonardo wants exactly. In our code, the main issue when it comes to optimality and time is about the randomness factor. But, it also makes the results more believable considering the hours will be examined by the HR. Other than this, we tried our code with various different inputs in order to check if it works correctly and concluded that it does. It is also easily maintable since the parameters are given by the user, so it is easy to change and play with the parameter values. If it was asked to improve the time complexity of the code, it is possible to get rid of the randomness. It could be done by:

* An algorithm which assigns the maximum (or minimum) hours for d-1 days and
assigning the sumHours - currentSum for the last day.
* An algorithm which assigns the mean of minimum and maximum hours for d-1 days and assigning the sumHours - currentSum for the last day.